In [ ]:
import folium
import pandas as pd

In [ ]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [ ]:
import requests
from io import StringIO
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

response = requests.get(URL)

spacex_df = pd.read_csv(StringIO(response.text))

spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [ ]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class', 'Flight Number']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [ ]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [ ]:
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [ ]:
for _, row in launch_sites_df.iterrows():
    # Extract coordinates (latitude, longitude) and launch site name
    coordinate = (row['Lat'], row['Long'])
    site_name = row['Launch Site']

    # Add a circle for each launch site
    circle = folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(site_name))
    site_map.add_child(circle)

    # Add a marker with an icon and the launch site name as a popup label
    marker = folium.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % site_name
        )
    )
    site_map.add_child(marker)

# Show the map
site_map

In [ ]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class,Flight Number
46,KSC LC-39A,28.573255,-80.646895,1,43
47,KSC LC-39A,28.573255,-80.646895,1,44
48,KSC LC-39A,28.573255,-80.646895,1,54
49,CCAFS SLC-40,28.563197,-80.576820,1,45
50,CCAFS SLC-40,28.563197,-80.576820,1,47
51,CCAFS SLC-40,28.563197,-80.576820,0,48
52,CCAFS SLC-40,28.563197,-80.576820,0,50
53,CCAFS SLC-40,28.563197,-80.576820,0,52
54,CCAFS SLC-40,28.563197,-80.576820,1,53
55,CCAFS SLC-40,28.563197,-80.576820,0,56


In [ ]:
marker_cluster = MarkerCluster()

In [ ]:
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')


for index, record in spacex_df.iterrows():
    coordinate = (record['Lat'], record['Long'])

    marker = folium.Marker(
        location=coordinate,
        icon=folium.Icon(color='white', icon_color=record['marker_color']),
        popup=f"Flight Number: {record['Flight Number']}, Outcome: {'Success' if record['class'] == 1 else 'Failure'}"
    )
    marker_cluster.add_child(marker)

site_map.add_child(marker_cluster)


site_map

In [ ]:
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [ ]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [ ]:
launch_site_lat = 28.56367  # Latitude of Kennedy Space Center
launch_site_lon = -80.57163  # Longitude of Kennedy Space Center
coastline_lat = 28.5623
coastline_lon = -80.5677

distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

coastline_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 12; color:#d35400;"><b>{distance_coastline:.2f} KM</b></div>'
    )
)
site_map.add_child(coastline_marker)

# Create and add a Marker for the launch site
launch_site_marker = folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup='Launch Site'
)
site_map.add_child(launch_site_marker)

# Draw a PolyLine between the launch site and coastline
coordinates = [(launch_site_lat, launch_site_lon), (coastline_lat, coastline_lon)]
lines = folium.PolyLine(locations=coordinates, weight=2, color='blue')
site_map.add_child(lines)

# Display the map
site_map